# Chapter 33: Advanced Prompt Engineering & Chain Techniques

Run this notebook directly in Google Colab - no local Python needed!

**Full code**: [GitHub](https://github.com/eduardd76/AI_for_networking_and_security_engineers/tree/main/CODE/Volume-3-Production-Systems/Chapter-33-Prompt-Engineering)

## Setup

Install dependencies and configure API keys.

In [ ]:
# Install dependencies
!pip install -q anthropic python-dotenv

# Import and configure API key
import os
from getpass import getpass

# Check for Colab secrets first
try:
    from google.colab import userdata
    os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
    print('✓ Using API keys from Colab secrets')
except:
    # Fall back to manual entry
    if 'ANTHROPIC_API_KEY' not in os.environ:
        os.environ['ANTHROPIC_API_KEY'] = getpass('Enter ANTHROPIC_API_KEY: ')
    print('✓ API keys configured')

print('\n✅ Setup complete! Ready to run examples.')

## Example 1: Basic Prompt Chaining

Decompose complex network analysis into sequential steps. Each step feeds into the next.

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

def call_claude(prompt, model="claude-sonnet-4-5-20250929"):
    """Single Claude API call"""
    message = client.messages.create(
        model=model,
        max_tokens=4096,
        messages=[{"role": "user", "content": prompt}]
    )
    return message.content[0].text

# Network config to analyze
CONFIG = """
interface GigabitEthernet0/1
 description UPLINK_TO_CORE
 ip address 10.1.1.2 255.255.255.252
 ip access-group EDGE-IN in
!
interface GigabitEthernet0/2
 description USER_VLAN_10
 ip address 192.168.10.1 255.255.255.0
 ip helper-address 10.2.2.10
!
ip access-list extended EDGE-IN
 permit tcp any any established
 permit udp any eq bootps any eq bootpc
 permit icmp any any echo-reply
 deny ip any any log
"""

# Step 1: Extract structure
prompt1 = f"""Analyze this network config and extract:
1. All interfaces with their IPs and descriptions
2. All ACLs and their rules
3. Any DHCP relay configurations

Config:
{CONFIG}

Output as structured JSON."""

result1 = call_claude(prompt1)
print("STEP 1 - Structure Extraction:")
print(result1)
print("\n" + "="*80 + "\n")

# Step 2: Security analysis using Step 1 output
prompt2 = f"""Given this parsed network configuration:

{result1}

Identify security issues:
1. Overly permissive ACL rules
2. Missing security controls
3. Potential attack vectors

Format as: Issue | Severity | Recommendation"""

result2 = call_claude(prompt2)
print("STEP 2 - Security Analysis:")
print(result2)
print("\n" + "="*80 + "\n")

# Step 3: Generate remediation config
prompt3 = f"""Based on these security findings:

{result2}

Generate Cisco IOS commands to fix the issues. Include:
1. Updated ACL entries
2. Additional security hardening
3. Verification commands

Use standard Cisco IOS syntax."""

result3 = call_claude(prompt3)
print("STEP 3 - Remediation Config:")
print(result3)

## Example 2: Chain-of-Thought Troubleshooting

Use chain-of-thought prompting to debug network issues with explicit reasoning steps.

In [ ]:
def troubleshoot_with_cot(symptom, topology, configs):
    """Use chain-of-thought to debug network issues"""

    prompt = f"""You are troubleshooting a network issue. Work through this step-by-step.

SYMPTOM:
{symptom}

TOPOLOGY:
{topology}

RELEVANT CONFIGS:
{configs}

Use this troubleshooting framework:

1. DEFINE THE PROBLEM
   - What is working vs. not working?
   - What traffic is affected?
   - Scope: single host, subnet, or site-wide?

2. ISOLATE THE LAYER
   - Is Layer 1/2 up? (interface status, CDP/LLDP)
   - Is Layer 3 routing correct? (routing table, ARP)
   - Is Layer 4-7 functional? (ACLs, NAT, firewall)

3. IDENTIFY THE ROOT CAUSE
   - What configuration or state is incorrect?
   - Why did it break? (change, failure, misconfiguration)

4. PROPOSE SOLUTION
   - Specific commands to fix
   - Verification steps
   - Rollback plan

Show your reasoning for each step. Think like a CCIE would troubleshoot this."""

    message = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )

    return message.content[0].text

# Real troubleshooting scenario
symptom = """
Users in VLAN 20 (192.168.20.0/24) cannot reach the internet.
Users in VLAN 10 (192.168.10.0/24) have no issues.
Both VLANs can ping their default gateway (10.1.1.1).
"""

topology = """
[VLAN 10: 192.168.10.0/24] ---\\
                                 [SW1] --- [RTR1: 10.1.1.1] --- [RTR2] --- [Internet]
[VLAN 20: 192.168.20.0/24] ---/
"""

configs = """
# RTR1 Config (relevant parts)
interface GigabitEthernet0/0.10
 encapsulation dot1Q 10
 ip address 192.168.10.1 255.255.255.0
 ip nat inside

interface GigabitEthernet0/0.20
 encapsulation dot1Q 20
 ip address 192.168.20.1 255.255.255.0
 ip nat inside

interface GigabitEthernet0/1
 ip address 10.1.1.1 255.255.255.252
 ip nat outside

ip nat inside source list NAT-ACL interface GigabitEthernet0/1 overload

ip access-list standard NAT-ACL
 permit 192.168.10.0 0.0.0.255

ip route 0.0.0.0 0.0.0.0 10.1.1.2
"""

result = troubleshoot_with_cot(symptom, topology, configs)
print("CHAIN-OF-THOUGHT TROUBLESHOOTING:")
print(result)

## Example 3: Few-Shot Config Translation

Use few-shot learning to translate configs between vendors with consistent format.

In [ ]:
def translate_config_fewshot(source_config, source_vendor, target_vendor):
    """Translate configs using few-shot examples"""

    # Few-shot examples showing exact translation patterns
    examples = """
Example 1:
Cisco IOS:
interface GigabitEthernet0/1
 description UPLINK
 ip address 10.1.1.1 255.255.255.0

Arista EOS:
interface Ethernet1
 description UPLINK
 ip address 10.1.1.1/24

Example 2:
Cisco IOS:
ip route 0.0.0.0 0.0.0.0 10.1.1.254

Arista EOS:
ip route 0.0.0.0/0 10.1.1.254

Example 3:
Cisco IOS:
router bgp 65001
 neighbor 10.1.1.2 remote-as 65002
 network 192.168.1.0 mask 255.255.255.0

Arista EOS:
router bgp 65001
 neighbor 10.1.1.2 remote-as 65002
 network 192.168.1.0/24
"""

    prompt = f"""Translate the following network configuration from {source_vendor} to {target_vendor}.

Use these examples as a guide for syntax differences:

{examples}

Now translate this configuration:

{source_config}

Provide only the translated configuration, no explanations."""

    message = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=4096,
        messages=[{"role": "user", "content": prompt}]
    )

    return message.content[0].text

# Test with a complex config
cisco_config = """
interface GigabitEthernet0/0
 description CORE_LINK
 ip address 10.10.10.1 255.255.255.252
 no shutdown

interface GigabitEthernet0/1
 description DMZ_SEGMENT
 ip address 172.16.1.1 255.255.255.0
 ip access-group DMZ-IN in

ip access-list extended DMZ-IN
 permit tcp any host 172.16.1.10 eq 443
 permit tcp any host 172.16.1.10 eq 80
 deny ip any any log

ip route 0.0.0.0 0.0.0.0 10.10.10.2

router bgp 65100
 bgp log-neighbor-changes
 neighbor 10.10.10.2 remote-as 65200
 network 172.16.1.0 mask 255.255.255.0
"""

result = translate_config_fewshot(cisco_config, "Cisco IOS", "Arista EOS")
print("CISCO IOS CONFIG:")
print(cisco_config)
print("\n" + "="*80)
print("\nARISTA EOS TRANSLATION:")
print(result)

## Example 4: Prompt Templates with Versioning

Build reusable, versioned prompt templates for consistent network operations.

In [ ]:
from datetime import datetime
from typing import Dict, Any

class PromptTemplate:
    """Versioned prompt templates for network operations"""

    def __init__(self, name: str, version: str, template: str, metadata: Dict[str, Any]):
        self.name = name
        self.version = version
        self.template = template
        self.metadata = metadata
        self.created = datetime.now()

    def render(self, **kwargs) -> str:
        """Render template with variables"""
        return self.template.format(**kwargs)

# Template library
TEMPLATES = {
    "acl_analysis": PromptTemplate(
        name="acl_analysis",
        version="2.1.0",
        template="""Analyze this ACL for security issues.

ACL Name: {acl_name}
ACL Type: {acl_type}
Applied Direction: {direction}
Interface: {interface}

ACL Rules:
{acl_rules}

Identify:
1. Overly permissive rules (any any patterns)
2. Shadowed rules (unreachable due to ordering)
3. Missing rules (common services not explicitly handled)
4. Log configurations (missing or excessive)
5. Best practice violations

Format findings as:
FINDING | SEVERITY | LINE | RECOMMENDATION""",
        metadata={
            "author": "Ed@vExpertAI",
            "tested_with": "claude-sonnet-4-5-20250929",
            "use_case": "ACL security auditing"
        }
    )
}

def execute_template(template_name: str, **kwargs) -> str:
    """Execute a template with given parameters"""
    template = TEMPLATES[template_name]
    prompt = template.render(**kwargs)

    print(f"Using Template: {template.name} v{template.version}")
    print(f"Author: {template.metadata['author']}")
    print(f"Use Case: {template.metadata['use_case']}")
    print("="*80)

    message = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=4096,
        messages=[{"role": "user", "content": prompt}]
    )

    return message.content[0].text

# Example: ACL Analysis
acl_output = execute_template(
    "acl_analysis",
    acl_name="INTERNET-EDGE",
    acl_type="extended",
    direction="in",
    interface="GigabitEthernet0/0",
    acl_rules="""10 permit tcp any any established
20 permit tcp any any eq 80
30 permit tcp any any eq 443
40 permit udp any any eq 53
50 deny ip any any log"""
)

print("\nACL ANALYSIS RESULT:")
print(acl_output)

## Interactive Section

Try your own prompt engineering experiments here!

In [ ]:
# Your code here
# Experiment with different prompts and network configs


## Next Steps

- Full code: [Chapter 33 on GitHub](https://github.com/eduardd76/AI_for_networking_and_security_engineers/tree/main/CODE/Volume-3-Production-Systems/Chapter-33-Prompt-Engineering)
- Learn more: [vExpertAI.com](https://vexpertai.com)
- Author: Eduard Dulharu ([@eduardd76](https://github.com/eduardd76))